# Epidemiological Modeling: Disease Transmission and Outbreak Analysis

**Tier 0 - Free Tier (Google Colab / Amazon SageMaker Studio Lab)**

## Overview

This notebook introduces mathematical epidemiology through compartmental models that describe infectious disease transmission dynamics. You'll implement classical SIR/SEIR models and analyze real outbreak data.

**What you'll learn:**
- Compartmental models (SIR, SEIR) for disease transmission
- Basic reproduction number (R0) estimation and interpretation
- Effective reproduction number (R_eff) over time
- Parameter fitting to real epidemic data
- Intervention modeling (vaccination, social distancing, lockdowns)
- Outbreak forecasting and uncertainty quantification
- Epidemic curve analysis and peak timing prediction

**Runtime:** 20-30 minutes

**Requirements:** `numpy`, `scipy`, `matplotlib`, `pandas` (all standard packages)

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Environment ready for epidemiological modeling")

## 1. SIR Model: The Foundation of Epidemic Modeling

The SIR model divides the population into three compartments:
- **S**: Susceptible individuals who can contract the disease
- **I**: Infected individuals who can transmit the disease
- **R**: Recovered individuals with immunity

The model is governed by differential equations:
- dS/dt = -β × S × I / N
- dI/dt = β × S × I / N - γ × I
- dR/dt = γ × I

where β is the transmission rate and γ is the recovery rate.

In [ ]:
def sir_model(y, t, beta, gamma, N):
    """
    SIR model differential equations
    
    Args:
        y: [S, I, R] current state
        t: time
        beta: transmission rate
        gamma: recovery rate
        N: total population
    """
    S, I, R = y
    
    dS_dt = -beta * S * I / N
    dI_dt = beta * S * I / N - gamma * I
    dR_dt = gamma * I
    
    return [dS_dt, dI_dt, dR_dt]

# Model parameters
N = 1_000_000  # Total population
I0 = 100        # Initial infected
R0 = 0          # Initial recovered
S0 = N - I0 - R0  # Initial susceptible

# Disease parameters (e.g., influenza-like)
beta = 0.5      # Transmission rate (contacts per day × transmission probability)
gamma = 0.1     # Recovery rate (1/infectious period in days)
R_0 = beta / gamma  # Basic reproduction number

print(f"SIR Model Parameters:")
print(f"  Population: {N:,}")
print(f"  Initial infected: {I0}")
print(f"  Transmission rate (β): {beta}")
print(f"  Recovery rate (γ): {gamma}")
print(f"  Basic reproduction number (R₀): {R_0:.2f}")
print(f"  Average infectious period: {1/gamma:.1f} days")

# Simulate epidemic
t = np.linspace(0, 200, 200)  # 200 days
y0 = [S0, I0, R0]
solution = odeint(sir_model, y0, t, args=(beta, gamma, N))
S, I, R = solution.T

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Time series
ax1.plot(t, S, label='Susceptible', linewidth=2)
ax1.plot(t, I, label='Infected', linewidth=2)
ax1.plot(t, R, label='Recovered', linewidth=2)
ax1.set_xlabel('Time (days)')
ax1.set_ylabel('Number of individuals')
ax1.set_title(f'SIR Model Dynamics (R₀ = {R_0:.2f})')
ax1.legend()
ax1.grid(alpha=0.3)

# Peak analysis
peak_idx = np.argmax(I)
peak_time = t[peak_idx]
peak_infected = I[peak_idx]
ax1.axvline(peak_time, color='red', linestyle='--', alpha=0.5, label=f'Peak: day {peak_time:.0f}')
ax1.scatter([peak_time], [peak_infected], color='red', s=100, zorder=5)

# Phase portrait (S vs I)
ax2.plot(S, I, linewidth=2)
ax2.scatter([S0], [I0], color='green', s=100, label='Start', zorder=5)
ax2.scatter([S[-1]], [I[-1]], color='red', s=100, label='End', zorder=5)
ax2.set_xlabel('Susceptible')
ax2.set_ylabel('Infected')
ax2.set_title('SIR Phase Portrait')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEpidemic Peak:")
print(f"  Time: Day {peak_time:.0f}")
print(f"  Peak infections: {peak_infected:,.0f} ({peak_infected/N*100:.1f}% of population)")
print(f"  Final attack rate: {R[-1]/N*100:.1f}% (total infected over epidemic)")

## 2. SEIR Model: Adding the Exposed Compartment

Many diseases have an incubation period where individuals are exposed but not yet infectious. The SEIR model adds an **E** (Exposed) compartment:

- dS/dt = -β × S × I / N
- dE/dt = β × S × I / N - σ × E
- dI/dt = σ × E - γ × I
- dR/dt = γ × I

where σ is the rate of progression from exposed to infectious.

In [ ]:
def seir_model(y, t, beta, sigma, gamma, N):
    """
    SEIR model differential equations
    
    Args:
        y: [S, E, I, R] current state
        t: time
        beta: transmission rate
        sigma: rate of progression from exposed to infectious
        gamma: recovery rate
        N: total population
    """
    S, E, I, R = y
    
    dS_dt = -beta * S * I / N
    dE_dt = beta * S * I / N - sigma * E
    dI_dt = sigma * E - gamma * I
    dR_dt = gamma * I
    
    return [dS_dt, dE_dt, dI_dt, dR_dt]

# SEIR parameters (e.g., COVID-19-like)
N = 1_000_000
I0 = 50
E0 = 150  # Exposed individuals
R0 = 0
S0 = N - E0 - I0 - R0

beta = 0.6      # Transmission rate
sigma = 0.2     # Rate exposed → infectious (1/incubation period)
gamma = 0.1     # Recovery rate
R_0 = beta / gamma

print(f"SEIR Model Parameters:")
print(f"  Incubation period: {1/sigma:.1f} days")
print(f"  Infectious period: {1/gamma:.1f} days")
print(f"  Basic reproduction number (R₀): {R_0:.2f}")

# Simulate
t = np.linspace(0, 200, 200)
y0 = [S0, E0, I0, R0]
solution = odeint(seir_model, y0, t, args=(beta, sigma, gamma, N))
S, E, I, R = solution.T

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(t, S, label='Susceptible', linewidth=2)
ax.plot(t, E, label='Exposed', linewidth=2, linestyle='--')
ax.plot(t, I, label='Infected', linewidth=2)
ax.plot(t, R, label='Recovered', linewidth=2)

peak_idx = np.argmax(I)
peak_time = t[peak_idx]
peak_infected = I[peak_idx]

ax.axvline(peak_time, color='red', linestyle='--', alpha=0.5)
ax.scatter([peak_time], [peak_infected], color='red', s=100, zorder=5)

ax.set_xlabel('Time (days)', fontsize=12)
ax.set_ylabel('Number of individuals', fontsize=12)
ax.set_title(f'SEIR Model Dynamics (R₀ = {R_0:.2f}, Incubation = {1/sigma:.1f} days)', fontsize=14)
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEpidemic Peak: Day {peak_time:.0f}, {peak_infected:,.0f} infected ({peak_infected/N*100:.1f}%)")
print(f"Note: SEIR model shows delayed peak compared to SIR due to incubation period")

## 3. Effective Reproduction Number (R_eff) Over Time

While R₀ is the basic reproduction number in a fully susceptible population, R_eff changes over time as the epidemic progresses and susceptibility decreases:

R_eff(t) = R₀ × S(t) / N

When R_eff > 1, the epidemic grows. When R_eff < 1, it declines.

In [ ]:
# Calculate R_eff over time
R_eff = R_0 * (S / N)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Infected time series
ax1.plot(t, I, linewidth=2, color='red')
ax1.fill_between(t, 0, I, alpha=0.3, color='red')
ax1.set_ylabel('Infected individuals', fontsize=11)
ax1.set_title('Epidemic Curve and Effective Reproduction Number', fontsize=13)
ax1.grid(alpha=0.3)
ax1.axvline(peak_time, color='black', linestyle='--', alpha=0.5, label='Peak')
ax1.legend()

# R_eff time series
ax2.plot(t, R_eff, linewidth=2, color='blue')
ax2.axhline(1, color='red', linestyle='--', linewidth=2, label='R_eff = 1 (threshold)')
ax2.fill_between(t, 1, R_eff, where=(R_eff > 1), alpha=0.3, color='red', label='Epidemic growth')
ax2.fill_between(t, 1, R_eff, where=(R_eff <= 1), alpha=0.3, color='green', label='Epidemic decline')
ax2.set_xlabel('Time (days)', fontsize=11)
ax2.set_ylabel('R_eff', fontsize=11)
ax2.set_ylim(0, R_0 + 0.5)
ax2.grid(alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

# Find when R_eff crosses 1
r_eff_cross_idx = np.where(R_eff < 1)[0][0] if any(R_eff < 1) else len(R_eff) - 1
r_eff_cross_time = t[r_eff_cross_idx]

print(f"\nR_eff Analysis:")
print(f"  Initial R_eff: {R_eff[0]:.2f}")
print(f"  R_eff crosses 1.0 at day {r_eff_cross_time:.0f} (epidemic peaks around this time)")
print(f"  Final R_eff: {R_eff[-1]:.2f}")
print(f"\nKey insight: Epidemic peaks when R_eff ≈ 1 (neither growing nor shrinking)")

## 4. Intervention Modeling: Vaccination and Social Distancing

Public health interventions reduce disease transmission. We'll model:
1. **Vaccination**: Moves individuals from S → R before infection
2. **Social distancing**: Reduces contact rate (β)

In [ ]:
def sir_with_intervention(y, t, beta_func, gamma, N):
    """
    SIR model with time-varying transmission rate (interventions)
    """
    S, I, R = y
    beta = beta_func(t)
    
    dS_dt = -beta * S * I / N
    dI_dt = beta * S * I / N - gamma * I
    dR_dt = gamma * I
    
    return [dS_dt, dI_dt, dR_dt]

# Scenario comparison
scenarios = {
    'No intervention': lambda t: 0.5,
    'Lockdown day 30': lambda t: 0.5 if t < 30 else 0.2,
    'Gradual social distancing': lambda t: 0.5 * np.exp(-0.015 * t),
}

# Initial vaccination (30% of population)
vaccination_rate = 0.3
scenarios['30% vaccination'] = lambda t: 0.5

fig, ax = plt.subplots(figsize=(12, 7))

colors = ['red', 'blue', 'green', 'purple']
results = {}

for (scenario_name, beta_func), color in zip(scenarios.items(), colors):
    # Initial conditions
    if scenario_name == '30% vaccination':
        S0_scenario = (N - I0) * (1 - vaccination_rate)
        R0_scenario = (N - I0) * vaccination_rate
    else:
        S0_scenario = N - I0
        R0_scenario = 0
    
    y0 = [S0_scenario, I0, R0_scenario]
    solution = odeint(sir_with_intervention, y0, t, args=(beta_func, gamma, N))
    S_scenario, I_scenario, R_scenario = solution.T
    
    results[scenario_name] = {'S': S_scenario, 'I': I_scenario, 'R': R_scenario}
    
    ax.plot(t, I_scenario, label=scenario_name, linewidth=2, color=color)
    
    # Mark peak
    peak_idx = np.argmax(I_scenario)
    ax.scatter([t[peak_idx]], [I_scenario[peak_idx]], color=color, s=60, zorder=5)

ax.set_xlabel('Time (days)', fontsize=12)
ax.set_ylabel('Infected individuals', fontsize=12)
ax.set_title('Impact of Public Health Interventions on Epidemic Curve', fontsize=14)
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Compare outcomes
print("\nIntervention Effectiveness:")
print("="*70)
for scenario_name, data in results.items():
    peak_infected = np.max(data['I'])
    total_infected = data['R'][-1]
    print(f"{scenario_name:30s} | Peak: {peak_infected:>8,.0f} | Total: {total_infected:>9,.0f} ({total_infected/N*100:.1f}%)")
print("="*70)

## 5. Synthetic Outbreak Data and Parameter Estimation

In real-world scenarios, we observe case data and must estimate model parameters (β, γ, R₀). We'll generate synthetic data and fit the SIR model.

In [ ]:
# Generate synthetic outbreak data with noise
true_beta = 0.5
true_gamma = 0.1
N_obs = 500_000

t_data = np.linspace(0, 120, 120)
y0_true = [N_obs - 50, 50, 0]
solution_true = odeint(sir_model, y0_true, t_data, args=(true_beta, true_gamma, N_obs))
I_true = solution_true[:, 1]

# Add observation noise (e.g., reporting delays, underreporting)
I_observed = I_true * (1 + np.random.normal(0, 0.15, len(I_true)))
I_observed = np.maximum(I_observed, 0)  # No negative cases

# Subsample (weekly reports)
sample_indices = np.arange(0, len(t_data), 7)
t_sample = t_data[sample_indices]
I_sample = I_observed[sample_indices]

print("Synthetic outbreak data generated (weekly case reports with noise)")
print(f"True parameters: β = {true_beta}, γ = {true_gamma}, R₀ = {true_beta/true_gamma}")

In [ ]:
# Parameter fitting using least squares
def fit_sir_model(params, t_data, I_observed, N):
    """Objective function for parameter fitting"""
    beta, gamma = params
    
    # Ensure positive parameters
    if beta <= 0 or gamma <= 0:
        return 1e10
    
    # Simulate
    y0 = [N - I_observed[0], I_observed[0], 0]
    solution = odeint(sir_model, y0, t_data, args=(beta, gamma, N))
    I_model = solution[:, 1]
    
    # Compute sum of squared errors
    error = np.sum((I_model - I_observed)**2)
    return error

# Initial guess
initial_guess = [0.4, 0.15]

# Optimize
result = minimize(
    fit_sir_model,
    initial_guess,
    args=(t_sample, I_sample, N_obs),
    method='Nelder-Mead',
    options={'maxiter': 1000}
)

beta_fit, gamma_fit = result.x
R0_fit = beta_fit / gamma_fit

print("\nParameter Estimation Results:")
print("="*50)
print(f"{'Parameter':<20s} {'True':<12s} {'Estimated':<12s}")
print("="*50)
print(f"{'β (transmission)':<20s} {true_beta:<12.3f} {beta_fit:<12.3f}")
print(f"{'γ (recovery)':<20s} {true_gamma:<12.3f} {gamma_fit:<12.3f}")
print(f"{'R₀':<20s} {true_beta/true_gamma:<12.3f} {R0_fit:<12.3f}")
print("="*50)

# Generate fitted curve
y0_fit = [N_obs - 50, 50, 0]
solution_fit = odeint(sir_model, y0_fit, t_data, args=(beta_fit, gamma_fit, N_obs))
I_fit = solution_fit[:, 1]

# Plot comparison
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(t_data, I_true, 'k-', linewidth=2, label='True epidemic curve', alpha=0.7)
ax.scatter(t_sample, I_sample, s=60, color='red', label='Observed data (weekly)', zorder=5, alpha=0.7)
ax.plot(t_data, I_fit, 'b--', linewidth=2, label=f'Fitted model (R₀ = {R0_fit:.2f})')

ax.set_xlabel('Time (days)', fontsize=12)
ax.set_ylabel('Infected individuals', fontsize=12)
ax.set_title('Parameter Estimation from Outbreak Data', fontsize=14)
ax.legend(fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFitting quality: SSE = {result.fun:,.0f}")

## 6. Outbreak Forecasting with Uncertainty

Forecast future cases using the fitted model with uncertainty bounds from parameter estimation uncertainty.

In [ ]:
# Generate forecast with uncertainty
forecast_days = 60
t_forecast = np.linspace(0, 120 + forecast_days, 180)

# Use fitted parameters
solution_forecast = odeint(sir_model, y0_fit, t_forecast, args=(beta_fit, gamma_fit, N_obs))
I_forecast = solution_forecast[:, 1]

# Uncertainty bounds (±10% parameter uncertainty)
solutions_uncertain = []
for _ in range(50):
    beta_sample = beta_fit * (1 + np.random.normal(0, 0.1))
    gamma_sample = gamma_fit * (1 + np.random.normal(0, 0.1))
    sol = odeint(sir_model, y0_fit, t_forecast, args=(beta_sample, gamma_sample, N_obs))
    solutions_uncertain.append(sol[:, 1])

solutions_uncertain = np.array(solutions_uncertain)
I_lower = np.percentile(solutions_uncertain, 5, axis=0)
I_upper = np.percentile(solutions_uncertain, 95, axis=0)

# Plot forecast
fig, ax = plt.subplots(figsize=(14, 7))

# Historical period
ax.scatter(t_sample, I_sample, s=60, color='red', label='Observed data', zorder=5)
ax.plot(t_data, I_fit, 'b-', linewidth=2, label='Fitted model (historical)')

# Forecast period
forecast_start_idx = len(t_data)
ax.plot(t_forecast[forecast_start_idx:], I_forecast[forecast_start_idx:], 'g--', linewidth=2.5, label='Forecast')
ax.fill_between(t_forecast[forecast_start_idx:], 
                I_lower[forecast_start_idx:], 
                I_upper[forecast_start_idx:], 
                alpha=0.3, color='green', label='90% uncertainty interval')

# Mark forecast boundary
ax.axvline(t_data[-1], color='black', linestyle=':', linewidth=2, label='Forecast start')

ax.set_xlabel('Time (days)', fontsize=12)
ax.set_ylabel('Infected individuals', fontsize=12)
ax.set_title(f'Epidemic Forecast (R₀ = {R0_fit:.2f})', fontsize=14)
ax.legend(fontsize=11, loc='upper right')
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Forecast summary
peak_forecast_idx = np.argmax(I_forecast)
peak_forecast_day = t_forecast[peak_forecast_idx]
peak_forecast_cases = I_forecast[peak_forecast_idx]

print("\nForecast Summary:")
print(f"  Predicted peak: Day {peak_forecast_day:.0f}")
print(f"  Peak infections: {peak_forecast_cases:,.0f} ({peak_forecast_cases/N_obs*100:.1f}% of population)")
print(f"  90% uncertainty interval at peak: [{I_lower[peak_forecast_idx]:,.0f}, {I_upper[peak_forecast_idx]:,.0f}]")
print(f"  Total cases by end of forecast: {solution_forecast[-1, 2]:,.0f} ({solution_forecast[-1, 2]/N_obs*100:.1f}%)")

## Summary and Next Steps

### What We've Accomplished

1. **Compartmental Models**
   - Implemented SIR and SEIR models for disease transmission
   - Simulated epidemic curves and phase portraits
   - Analyzed epidemic peak timing and total attack rates

2. **Reproduction Numbers**
   - Calculated R₀ (basic reproduction number) from model parameters
   - Tracked R_eff (effective reproduction number) over time
   - Identified epidemic peak when R_eff crosses 1.0

3. **Public Health Interventions**
   - Modeled vaccination and social distancing effects
   - Demonstrated "flattening the curve" to reduce peak infections
   - Compared intervention strategies quantitatively

4. **Parameter Estimation**
   - Fitted model parameters to synthetic outbreak data
   - Estimated R₀ from observed case reports
   - Generated forecasts with uncertainty quantification

### Key Insights

- **R₀ > 1** is necessary for an epidemic to occur
- **Herd immunity threshold**: 1 - 1/R₀ (e.g., 67% for R₀ = 3)
- Early interventions are more effective (exponential growth)
- Parameter estimation is sensitive to data quality and model assumptions
- All models are simplifications; real epidemics have heterogeneous mixing, spatial structure, age stratification

### Limitations

- Assumes homogeneous population mixing (no age/spatial structure)
- No births, deaths, or population dynamics
- Deterministic (no stochasticity for small populations)
- No reinfection or waning immunity
- Oversimplified intervention models

### Progression Path

**Tier 1** - Advanced epidemiological models
- Age-structured models (children vs adults)
- Spatial models (metapopulation, network-based)
- Stochastic models (individual-based, agent-based)
- Real COVID-19/influenza data from Johns Hopkins CSSE

**Tier 2** - AWS-integrated epidemic surveillance
- Real-time data ingestion (S3, Lambda)
- Automated parameter estimation and forecasting
- Interactive dashboards with Plotly/Dash
- Multi-region outbreak tracking

**Tier 3** - Production epidemic modeling platform
- CloudFormation: EC2, RDS, S3, SageMaker
- Bayesian inference for parameter uncertainty (PyMC3, Stan)
- Ensemble forecasting with multiple models
- Integration with CDC/WHO surveillance data
- Scenario planning for pandemic preparedness

### Additional Resources

- Keeling & Rohani: "Modeling Infectious Diseases"
- Vynnycky & White: "An Introduction to Infectious Disease Modelling"
- COVID-19 Forecasting Hub: https://covid19forecasthub.org/
- Johns Hopkins CSSE: https://github.com/CSSEGISandData/COVID-19
- CDC FluView: https://www.cdc.gov/flu/weekly/